# Processing & Loading data from government public dataset api to mongodb database using spark connector

## Import necessary packages

In [44]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql import functions as func

## Get request to the government public dataset 

In [45]:
request= requests.get("https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69?api-key=579b464db66ec23bdd00000194a8bf20f8ec499241a734856f3a3504&format=json")

In [46]:
json_str=request.content

## Load JSON byte string into python dictionary

In [47]:
data_dict=json.loads(json_str)

In [48]:
print(data_dict)

{'index_name': '3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69', 'title': 'Real time Air Quality Index from various locations', 'desc': 'Real time Air Quality Index from various locations', 'org_type': 'Central', 'org': ['Ministry of Environment, Forest and Climate Change', 'Central Pollution Control Board'], 'sector': ['Environment and Forest', 'Vehicular Air Pollution', 'Residential Air Pollution', 'Industrial Air Pollution'], 'source': 'data.gov.in', 'catalog_uuid': 'a3e7afc6-b799-4ede-b143-8e074b27e062', 'visualizable': '0', 'active': 1, 'created': '1543320551', 'updated': 1658642526, 'created_date': '2018-11-27T17:39:11Z', 'updated_date': '2022-07-24T11:32:06Z', 'external_ws': 0, 'external_ws_url': '', 'target_bucket': {'index': 'aqi_archieve', 'type': 'a3e7afc6-b799-4ede-b143-8e074b27e062', 'field': '3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69'}, 'field': [{'id': 'id', 'name': 'id', 'type': 'double'}, {'id': 'country', 'name': 'country', 'type': 'keyword'}, {'id': 'state', 'name': 'state', 'type'

In [49]:
dataset=data_dict['records']

## Define and configure sparksession
### Configuring mongo spark connector

In [50]:
spark = SparkSession.builder.appName("air_quality").config("spark.mongodb.input.uri", "mongodb://127.0.0.1:49752/public_db.air_quality") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1:49752/public_db.air_quality")\
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector:10.0.2").getOrCreate()

In [51]:
df_json = spark.createDataFrame(dataset)

In [52]:
df_final=df_json.drop("pollutant_unit")

### Writing the DataFrame to mongoDB
#### Database: public_db; Collections: air_quality

In [53]:
df_final.write.format("mongodb").option("spark.mongodb.output.uri", "mongodb://127.0.0.1:49752/public_db.air_quality").option("database","public_db").option("collection","air_quality").mode("append").save()